Go back to the `SolveAiyagari.ipynb` notebook.

# Useful functions

In [1]:
function norm(x; p=nothing) 
    return isnothing(p) ? maximum(abs.(x)) : sum(abs.(x).^p)
end

norm (generic function with 1 method)

## Gini coefficient

We consider a discrete distribution of variables $\{y_i\}_{i=1,\ldots,n}$ with probabilities $\{p_{y_i}\}_{i=1,\ldots,n}$. The variables $\{y_i\}$ are positive and sorted in increasing order ($0<y_i<y_{i+1}$). 
The Gini coefficient $G$ is then defined as:
$$G = 1- \frac{\sum_{i=1}^n p_{y_i}(S_{i-1}+S_{i})}{S_n},$$
where $S_0=0$ and:
$$S_i = \sum_{j=1}^i y_jp_{y_j}.$$

The function we define is `Gini(ys, pys)`,where `ys` corresponds to $\{y_i\}_{i=1,\ldots,n}$, and `pys` to $\{p_{y_i}\}_{i=1,\ldots,n}$. The inputs `ys` and `pys` can be either of type `Vector` or `Matrix` (but both of the same type).

The input `ys` is *not* required to be sorted.

In [27]:
function Gini(ys::Vector{T}, pys::Vector{T})::T where{T<:Real}
    @assert size(ys)==size(pys)
    iys = sortperm(ys)
    ys .= ys[iys]
    pys .= pys[iys]
    Ss = [zero(T); cumsum(ys.*pys)]
    return one(T) - sum(pys.*(Ss[1:end-1].+Ss[2:end]))/Ss[end]
end
function Gini(ys::Matrix{T}, pys::Matrix{T})::T where{T<:Real}
    @assert size(ys)==size(pys)
    return Gini(ys[:],pys[:])
end

Gini (generic function with 2 methods)

This function computes the distribution by quintile

In [ ]:

function wealthD(ww::AbstractArray,NN::Integer)
    ww2 = sortslices(ww, dims=1)
    fracw = cumsum(ww2[:,1].*ww2[:,2])
    fracw =fracw ./fracw[end]
    fracS = cumsum(ww2[:,2])
    plot(fracS,fracw)
    vect = zeros(Float64,NN,2)
    is  = 0
    for i=1:NN
        is = maximum(findall(x->x<=i/NN,fracS))
        vect[i,1] = fracS[is] #- vect[i-1,1]
        vect[i,2] = fracw[is] #- vect[i-1,2]
    end
    vecf = zeros(Float64,NN,2)
    vecf[1,:] .= vect[1,:]
    for i=2:NN   
        vecf[i,1] = vect[i,1] - vect[i-1,1]
        vecf[i,2] = vect[i,2] - vect[i-1,2]
    end
    return  vecf
    end

In [ ]:
# Aigar

# Deprecated functions

These functions are not used anymore but kept for legacy reasons.

## Converting from base `E` to base 10 (and vice-versa)


### From base `E` to base 10

The function `convertBasisE10(vE, E, N)` converts a number expressed in base-`E` (and represented as the vector `vE`) into its decimal representation `p` (as an integer).

***Remarks*** 
* elements in `vE` are  between 1 and $E$ (and not 0 and $E-1$);
* `vE[1]` represents the largest power and `vE[N]` represents units.

In [3]:
function convertBasisE10(vE::Vector{I}, E::I)::I where {I<:Integer}
# converts the a vector vE into an integer p 

    @assert (minimum(vE)≥1)&&(maximum(vE)≤E)
    
    N = length(vE)    
    p = one(I) #otherwise p lieas between 0 and (E+1)^N - 1
    for k ∈ eachindex(vE)
        p += (vE[k]-1)*E^(N-k)
    end
    return p
end;

#### Example

We convert `[2, 2, 1]` (i.e., 110 with a more usual binary notation) into base 10.

In [7]:
convertBasisE10([2, 2, 1], 2)

7

### From base 10 to base `E`

The function `convertBasis10E(p, E, N)` converts a number expressed in base-10 (and represented as the integer p) into its base-`E` representation `vE` (as a vector of length `N`). This is the inverse of `convertBasisE10`.

Same remarks as for `convertBasisE10`.

In [10]:
function  convertBasis10E(p::I, E::I, N::I)::Vector{I} where{I<:Integer}

    vE = zeros(I, N)    
    @assert (p≥1)&&(p≤E^N)
    ptemp = p-1;
    for k ∈ eachindex(vE)
        ptemp, r = divrem(ptemp,E)
        vE[N-k+1] = 1+r        
    end
    return vE
end

convertBasis10E (generic function with 1 method)

#### Example

We check that 7 converts back into  `[2, 2, 1]`.

In [13]:
convertBasis10E(7, 2, 3)

3-element Vector{Int64}:
 2
 2
 1